In [1]:
"""
Feature: neurological_disorder
Extract from diagnoses_icd using ICD codes
"""
import pandas as pd
import os
from config_paths import *
from utils import drop_columns

In [2]:
# Read dataset
df = pd.read_csv(dataset_path)
initial_row_count = len(df)

# Load diagnoses
diagnoses = pd.read_csv(
    os.path.join(hosp_path, "diagnoses_icd.csv"),
    usecols=["subject_id", "hadm_id", "icd_code", "icd_version"]
)
diagnoses["icd_code"] = diagnoses["icd_code"].astype(str)

# Function to check if any ICD code matches patterns
def has_icd(icd_list, patterns):
    return icd_list.str.startswith(tuple(patterns)).any()

# Neurological disorder prefixes
neuro_prefixes = [
    "332", "340", "345", "353", "354", "355", "356", "357", "G"
]

neurological_df = (
    diagnoses
    .groupby(["subject_id", "hadm_id"])
    .apply(
        lambda g: pd.Series({
            "neurological_disorder": has_icd(g["icd_code"], neuro_prefixes)
        }),
        include_groups=False
    )
    .reset_index()
)

# Merge with dataset
df = df.merge(
    neurological_df,
    on=["subject_id", "hadm_id"],
    how="left"
)

# Fill missing values with False
df["neurological_disorder"] = df["neurological_disorder"].fillna(False).astype(bool)

# Save
df.to_csv(dataset_path, index=False)

# Diagnostic output
print(f"\nTotal rows: {len(df)}")
print(f"Expected total: {initial_row_count}")
if len(df) == initial_row_count:
    print(f"✓ Row count matches expected total ({initial_row_count})")
else:
    print(f"⚠ Row count mismatch! Expected {initial_row_count}, got {len(df)} (difference: {len(df) - initial_row_count})")
print(f"Dataset shape: {df.shape}")
print(f"Feature 'neurological_disorder' added. True count: {df['neurological_disorder'].sum()}")
print(f"Dataset shape: {df.shape}")


C:\Users\Coditas\AppData\Local\Temp\ipykernel_20360\1368148025.py:41: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["neurological_disorder"] = df["neurological_disorder"].fillna(False).astype(bool)



Total rows: 158020
Expected total: 158020
✓ Row count matches expected total (158020)
Dataset shape: (158020, 47)
Feature 'neurological_disorder' added. True count: 47096
Dataset shape: (158020, 47)


In [3]:
df.columns

Index(['subject_id', 'hadm_id', 'cauti_type', 'cauti_icd_codes',
       'other_uti_icd_codes', 'remaining_icd_codes', 'gender', 'anchor_age',
       'admittime', 'dischtime', 'admission_type', 'admission_location',
       'discharge_location', 'race', 'catheter_procedures',
       'catheter_procedures_ir', 'catheter_used_in_procedures_ir',
       'catheter_insertion_date_ir', 'catheter_removal_date_ir',
       'catheter_outputevents', 'catheter_used_in_output_events',
       'datetimeevents', 'catheter_used_in_datetime_events',
       'insertion_date_from_datetimeevents',
       'removal_date_from_datetimeevents', 'catheter_procedure_events',
       'catheter_used_in_procedure_events', 'catheter_size_from_chartevents',
       'index', 'catheter_clinical_notes', 'catheter_used_in_clinical_notes',
       'catheter_present', 'final_cauti_flag', 'final_insertion_date',
       'final_removal_date', 'catheter_duration_days', 'BMI_in_admission',
       'BMI_computed', 'BMI_last_year', 'BMI', 

In [5]:
# cols_to_drop = [
#  "neurological_disorder"
# ]
# df = drop_columns(df, cols_to_drop)
# df.to_csv(dataset_path, index=False)